# Model run: Forecast 
Use this notebook to run and evaluate a parameter grid-search. 

This notebook can be run after 0, 1, 2, and 3b. We recommend also running 3a first, to check for and troubleshoot issues.

## Set up workspace from env and configuration files 

First, import needed packages.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import dotenv
import os 
import json 

Navigate to main repository.

In [ ]:
os.chdir('../')

Import needed PoPS Global functions.

In [ ]:
from pandemic.multirun_helpers import write_commands, generate_param_samples
# import summary stats run 

Read in path variables from .env.

In [ ]:
# Load variables and paths from .env
dotenv.load_dotenv('.env')

# Read environmental variables
input_dir = os.getenv('INPUT_PATH')
out_dir = os.getenv('OUTPUT_PATH')


Read in parameters from config.json

In [ ]:
with open('config.json') as json_file:
    config = json.load(json_file)

sim_name = config['sim_name']
coi = config['coi']
sim_years = config['sim_years']

run_name = f"{sim_name}_calibrate"


## Use the summary stats from the grid search to generate a parameter distribution

Read summary statistics from file.

In [ ]:
stats_dir = f"{out_dir}/summary_stats/{run_name}"

col_dict = {"start_max":"start","alpha_max":"alpha","beta_max":"beta",
    "lamda_max":"lamda","count_known_countries_time_window_fbeta_mean":"fbeta"}

agg_df = (
    pd.read_csv(f"{stats_dir}/summary_stats_bySample.csv")
    .rename(columns=col_dict)
    )
    

### Set a performance threshold
Select a threhold percentile value (on F-beta) to determine which samples to use to fit the distribution. 

The viusalizations below help explore the possible thresholds and their impact on the number of samples included and the 
corresponding cut-off value for F-beta, and on the distribution of parameters included. 

In [ ]:
# Set a % threshold (0 - 100) - adjust based on the below plots

quant_threshold = 90

In [ ]:
count_vals = []
min_fbeta = []

for val in range(70,100):
    subset = agg_df.loc[agg_df['fbeta']>=agg_df['fbeta'].quantile(val/100)]
    count_vals.append(len(subset.index))
    min_fbeta.append(subset['fbeta'].min())
    

In [ ]:
sample_stats = pd.DataFrame(
    {"quantile":range(70,100), 
    "count":count_vals, 
    "min_fbeta":min_fbeta}
    ).set_index("quantile")
    

Visual: How many samples and what Fbeta scores are captured with each threshold?

In [ ]:
fig, (ax1, ax2, ) = plt.subplots(1, 2, figsize=(10, 5))
sample_stats["count"].plot(ax = ax1)
ax1.vlines(quant_threshold, ymin=sample_stats["count"].min(), 
    ymax=sample_stats["count"].max(), linestyle='dashed', color="firebrick")
ax1.set_title("Count")

sample_stats["min_fbeta"].plot(ax = ax2)
ax2.vlines(quant_threshold, ymin=sample_stats["min_fbeta"].min(), 
    ymax=sample_stats["min_fbeta"].max(), linestyle='dashed', color="firebrick")
ax2.set_title("Fbeta")

plt.show()

Visual: What do the distributions of alpha and lamda look like with that threshold?

In [ ]:
agg_df['top'] = np.where(agg_df['fbeta']>=agg_df['fbeta'].quantile(quant_threshold/100),'top','low')

Visualize separation by parameter.

In [ ]:
# Alpha by year

ax = sns.relplot(x="alpha",y="fbeta", 
    col="start",hue="top",palette="rocket",
    data=agg_df,edgecolor="black",linewidth=0.5,s=100)
    
plt.show()

In [ ]:
# Lamda by year

ax = sns.relplot(x="lamda",y="fbeta", 
    col="start",hue="top",palette="rocket",
    data=agg_df,edgecolor="black",linewidth=0.5,s=100)

plt.show()

In [ ]:
# Beta by year

ax = sns.relplot(x="beta",y="fbeta", 
    col="start",hue="top",palette="rocket",
    data=agg_df,edgecolor="black",linewidth=0.5,s=100)
    
plt.show()

Visualize overall parameter distributions of sampled sets. 

In [ ]:
# Top parameter distribution plot

ax = sns.relplot(x="alpha", y="lamda", 
    col="start", hue="fbeta", palette="mako_r", 
    data=agg_df.loc[agg_df['top']=="top"])
    
plt.show()

### Generate a multivariate normal distribution and sampled parameters
Using the samples above your threshold, randomly sample a set of new parameter sets from their distribution. 

In [ ]:
# How many distinct parameter samples do you want to generate?
n_samples = 1000


In [ ]:
# Fits a separate distribution per year 

samples_to_run = generate_param_samples(agg_df, n_samples)


In [ ]:
# Save sampled parameters to .csv as a backup/for later use

samples_to_run.to_csv(f"{stats_dir}/sampled_param_sets.csv")


Visualize the parameter distributions that will be run.

In [ ]:
# Plot to visually examine the parameter posterior distributions

ax = sns.jointplot(x="alpha", y="lamda", hue="start", data=samples_to_run, palette="deep", alpha = 0.4)
plt.show()


## Run the model forecast

First write out the commands with the new sampled parameter sets. One run will be conducted with each parameter sample.

In [ ]:
commands_forecast = ""

for index, row in samples_to_run.iterrows():
    commands_forecast += write_commands(row, start_run = 0, end_run = 0, run_type = "forecast")


In [ ]:
# # If you will run on HPC or later, write these to file 

# f1 = open(stats_dir + "/commands.txt", 'w')
# f1.write(commands_forecast)
# f1.close()


Run the cell below to execute all model runs. These must complete before you can calculate 
the summary statistics. Remember that this may take some time (approximately 2 - 5 minutes 
per run per core, depending on your computer and number of time-steps in your simulation), 
so prepare accordingly!


In [ ]:
# Run model here
for command in commands_forecast.split('\n'):
    ! {command}


These runs will write out to "outputs/{run_name}_forecast/". 

Calculate summary statistics on completed runs. This is also run in parallel, so time 
will vary depending on how many cores you use.

In [ ]:
# Generate summary stats

! python pandemic/get_stats.py forecast

## Review model summary statistics

You can summarize the model runs now with a single set of summary statistics. 

In [ ]:
run_name = f"{sim_name}_forecast"
stats_dir = f"{out_dir}/summary_stats/{run_name}"

agg_df = (
    pd.read_csv(f"{stats_dir}/summary_stats_bySample.csv")
    .rename(columns=col_dict)
    )

agg_df

In [ ]:
print(
    f"Final forecast summary results:"
    f"F-beta = {agg_df.loc[0,'fbeta']}"
    f"Probability of intro. to {coi} by {sim_years}: {agg_df.loc[0, [col for col in agg_df.columns if 'prob_by_' in col]]}")

## Next step: Visualize forecast

Use notebook 4 to visualize the full results of your forecast simulation. 